In [ ]:
from sctoolbox.utils.jupyter import bgcolor

# Gene Set Enrichment Analysis (GSEA)
<hr style="border:2px solid black"> </hr>

## 1 - Description

**Note: You need to have run the marker gene notebook before using the GSEA notebook**

The main function of this notebook is to get the enrichted GO pathways per cluster. For this we use the marker genes as the gene set input.  
This notebook uses [enrichr](https://maayanlab.cloud/Enrichr/) which is implemented in [geseapy](https://github.com/zqfang/GSEApy).    

`The enrichr module enable you perform gene set enrichment analysis using Enrichr API. Enrichr is open source and freely available online at: http://amp.pharm.mssm.edu/Enrichr . It runs very fast.` - GSEApy docu

---

## 2 - Setup

In [ ]:
import sctoolbox.utils as utils
import sctoolbox.tools as tools
from sctoolbox import settings

import pandas as pd
import gseapy as gp
import tqdm
import matplotlib.pyplot as plt

---

## 3 - General Input

In [ ]:
%bgcolor PowderBlue

# sctoolbox settings
settings.adata_input_dir = "../rna_analysis/adatas/"
settings.adata_output_dir = "../rna_analysis/adatas/"
settings.figure_dir = "../figures/GSEA/"
settings.log_file = "../logs/GSEA_log.txt"
last_notebook_adata = "anndata_5.h5ad"

organism = "human"

# key for marker table in adata.uns
marker_key = "rank_genes_leiden_0.1_filtered" 
pvals_adj_tresh = 0.05

---

## 4 - Load anndata

In [ ]:
adata = utils.adata.load_h5ad(last_notebook_adata)

with pd.option_context("display.max.rows", 5, "display.max.columns", None):
    display(adata)
    display(adata.obs)
    display(adata.var)

---

## 5 - Select library

In [ ]:
[db for db in gp.get_library_name(organism) if db.startswith("GO")]

In [ ]:
%bgcolor PowderBlue

library_name = "GO_Biological_Process_2023"

In [ ]:
## download a library or read a .gmt file
go_mf = gp.get_library(name=library_name, organism=organism)
# list of all genes as background
flat_list = set([item for sublist in go_mf.values() for item in sublist])

---

## 7 - Run enrichr

In [ ]:
combined = tools.gsea.enrichr_marker_genes(adata,
                                           marker_key = marker_key,
                                           gene_sets = go_mf,
                                           organism = organsim,
                                           background = flat_list,
                                           pvals_adj_tresh = pvals_adj_tresh)

---

## 8 - Plotting

In [ ]:
%bgcolor PowderBlue

# Dotplot
figsize = (8, 20)
top_term = 10

size = 5

In [ ]:
for reg in ["UP", "DOWN"]:
    comb = combined[combined["UP_DW"] == reg]
    if not comb.empty:
        ax = gp.dotplot(comb,
                        figsize=figsize,
                        x='Cluster',
                        title=f"Top {top_term} {reg} regulated Pathways per Cluster",
                        cmap = plt.cm.autumn_r,
                        size=size,
                        show_ring=True,
                        top_term=top_term,
                        xticklabels_rot=45
                       )
        ax.set_xlabel("")
        plt.tight_layout()
        plt.savefig(f"{settings.figure_dir}/GSEA_dotplot_top{top_term}_{reg}_pathways_per_cluster.pdf", dpi=300)

---